In [3]:
import pandas as pd
import numpy as np

In [45]:
path = "/Users/apple/PROJECT/Code_4_SECfilings/sp500_total_constituents.csv"
df_total = pd.read_csv(path)
df_total = df_total.rename(columns={"Symbol": "Ticker"})

path1 = "/Users/apple/PROJECT/Code_4_SECfilings/sp500_total_constituents2.csv"
df_total2 = pd.read_csv(path1)
df_total2 = df_total2.rename(columns={"Symbol": "Ticker"})

path2 = "/Users/apple/PROJECT/Code_4_SECfilings/sp500_2025_weights.csv"
df_2025 = pd.read_csv(path2)

path3 = "/Users/apple/PROJECT/Code_4_SECfilings/sp500_constituents.csv"
df_ori = pd.read_csv(path3)
df_ori = df_ori.rename(columns={"ticker": "Ticker"})
# Convert date columns to datetime format
df_ori['start'] = pd.to_datetime(df_ori['start'], errors='coerce')
df_ori['ending'] = pd.to_datetime(df_ori['ending'], errors='coerce')
df_ori['nameendt'] = pd.to_datetime(df_ori['nameendt'], errors='coerce')  # Ensure nameendt is datetime

path4 = "/Users/apple/PROJECT/Code_4_SECfilings/sp500_by_year.parquet"
df_year = pd.read_parquet(path4)





In [3]:
df_merge = df_total.merge(df_2025, on="Ticker", how="inner")

In [46]:
df_total_list = df_total["Ticker"].tolist()
df_2025_list = df_2025["Ticker"].tolist()
interect_total_2025 = list(set(df_total_list).intersection(df_2025_list))
len(interect_total_2025)


413

In [47]:
only_2025 = set(df_2025_list) - set(interect_total_2025)
print(len(only_2025))
with open("only_2025.txt", "w") as f:
    f.write(",".join(map(str, only_2025)))

90


In [48]:
only_total = set(df_total_list) - set(interect_total_2025)
with open("only_total.txt", "w") as f:
    f.write(",".join(map(str, only_total)))

In [69]:
from datetime import datetime
year = 2023
start_of_year = datetime(year, 1, 1) # 2006-01-01
end_of_year = datetime(year, 12, 31) # 2006-12-31
print(end_of_year)

2023-12-31 00:00:00


In [73]:
active_firms = df_ori[
        (df_ori['start'] <= end_of_year) &  (df_ori['ending'] >= start_of_year)
    ]
# Group by 'permno' and get the last entry based on 'nameendt'
active_firms = active_firms.sort_values(by=['permno', 'nameendt']).groupby('permno').last().reset_index()
print(active_firms.shape[0])
active_firms
active_firms[active_firms["Ticker"] == "ABNB"]
# active_firms = active_firms.drop_duplicates(subset=['permno'])
# print("active firms after drop_duplicates:", active_firms.shape[0])
# active_firms = active_firms[active_firms["Ticker"] == "ELV"]
# active_firms

518


,permno,Unnamed: 0,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,Ticker
117,20190,1078260,2023-09-18,2023-12-29,2023-12-29,0.077569,AIRBNB INC,00906610,2021-03-31,2023-12-29,11,3,9999,ABNB


In [36]:
df_ori_temp = df_ori[df_ori["Ticker"] == "ELV"]
active_firms_temp = df_ori_temp[
        (df_ori_temp['start'] <= end_of_year) &  (df_ori_temp['ending'] >= start_of_year)
    ]
print(active_firms_temp[active_firms_temp["Ticker"] == "ELV"].shape[0])
active_firms_temp = active_firms_temp.drop_duplicates(subset=['permno'])
active_firms_temp = active_firms_temp[active_firms_temp["Ticker"] == "ELV"]
active_firms_temp

19


,Unnamed: 0,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,Ticker
134946,1013119,89179,2002-07-25,2023-12-29,2022-06-30,-0.050529,ELEVANCE HEALTH INC,03675210,2022-06-28,2023-12-29,11,1,6321,ELV


In [28]:
df_ori_temp = df_ori[df_ori["Ticker"] == "ELV"]

condition = df_ori_temp['start'] <= end_of_year

positive_row = df_ori_temp[condition]
print(positive_row.head())
false_rows = df_ori_temp[~condition]



        Unnamed: 0  permno      start     ending        date       ret  \
134946     1013119   89179 2002-07-25 2023-12-29  2022-06-30 -0.050529   
135450     1016676   89179 2002-07-25 2023-12-29  2022-07-29 -0.011356   
136261     1022468   89179 2002-07-25 2023-12-29  2022-08-31  0.016789   
136930     1027129   89179 2002-07-25 2023-12-29  2022-09-30 -0.060997   
136975     1027473   89179 2002-07-25 2023-12-29  2022-10-31  0.203703   

                     comnam    ncusip      namedt    nameendt  shrcd  exchcd  \
134946  ELEVANCE HEALTH INC  03675210  2022-06-28  2023-12-29     11       1   
135450  ELEVANCE HEALTH INC  03675210  2022-06-28  2023-12-29     11       1   
136261  ELEVANCE HEALTH INC  03675210  2022-06-28  2023-12-29     11       1   
136930  ELEVANCE HEALTH INC  03675210  2022-06-28  2023-12-29     11       1   
136975  ELEVANCE HEALTH INC  03675210  2022-06-28  2023-12-29     11       1   

        hsiccd Ticker  
134946    6321    ELV  
135450    6321    ELV  
13

In [18]:
df_ori_temp = df_ori[df_ori["Ticker"] == "ELV"]
condition = df_ori_temp['ending'] >= start_of_year
false_rows = df_ori_temp[~condition]
print(false_rows.head())


Empty DataFrame
Columns: [Unnamed: 0, permno, start, ending, date, ret, comnam, ncusip, namedt, nameendt, shrcd, exchcd, hsiccd, Ticker]
Index: []


In [15]:
condition = df_ori['start'] <= end_of_year
false_rows = df_ori[~condition]
tickers_1 = false_rows["Ticker"].unique()
print(tickers_1)
if "SE" in tickers_1:
    print("None")


['AVB' 'SE' 'ESV' 'TEG' 'RL' 'VAR' 'HCBK' 'CHRW' 'DDR' 'HST' 'ANF' 'AIZ'
 'KFT' 'WFR' 'PCP' 'TEL' 'GGP' 'AKAM' 'DFS' 'ACAS' 'COV' 'LUK' 'TSO'
 'MCHP' 'ICE' 'EXPD' 'TIE' 'NBL' 'JEC' 'TDC' 'EXPE' 'NYX' 'AMT' 'POM'
 'MTW' 'RRC' 'WPO' 'GME' 'CAM' 'TSS' 'HCP' 'PM' 'COG' 'LO' 'SWN' 'MEE'
 'ISRG' 'DVA' 'MA' 'SNI' 'AKS' 'IVZ' 'CF' 'HRS' 'FAST' 'PXD' 'APH' 'CRM'
 'WEC' 'DPS' 'FLS' 'NDAQ' 'LIFE' 'CEPH' 'SRCL' 'PBCT' 'XRAY' 'SJM' 'WYNN'
 'DNB' 'MFE' 'EQT' 'RSG' 'HCN' 'SCG' 'OI' 'IRM' 'FLIR' 'DO' 'TWC' 'HRL'
 'ORLY' 'VTR' 'NU' 'DNR' 'DV' 'FTI' 'PCS' 'RHT' 'PWR' 'WDC' 'FMC' 'ARG'
 'CFN' 'FSLR' 'PCLN' 'MJN' 'V' 'ROST' 'SAI' 'CLF' 'ROP' 'NRG' 'URBN' 'BRK'
 'OKE' 'DISCA' 'HP' 'CERN' 'KMX' 'QEP' 'ACE' 'TYC' 'IR' 'NFX' 'CVC' 'NFLX'
 'FFIV' 'MMI' 'NE' 'JOYG' 'CMG' 'BLK' 'EW' 'PLD' 'ANR' 'MPC' 'ACN' 'MOS'
 'XYL' 'CBE' 'JOY' 'DLTR' 'PRGO' 'TRIP' 'GAS' 'BWA' 'WPX' 'CCI' 'FOSL'
 'PSX' 'KMI' 'ALXN' 'MNST' 'LRCX' 'STX' 'LYB' 'PETM' 'MDLZ' 'KRFT' 'PNR'
 'ADT' 'DG' 'DLPH' 'GRMN' 'ABBV' 'PVH' 'MAC' 'REGN' 'KSU' '

In [86]:
condition = df_ori['ending'] >= start_of_year
false_rows = df_ori[~condition]
tickers_2 = false_rows["Ticker"].unique()
print(tickers_2)
if "ELV" in tickers_2:
    print("None")

['U' 'UPC' 'PCH' 'MDR' 'FBF' 'RAD' 'RD' 'T' 'UCM' 'WLA' 'TNB' 'WCOM' 'PWJ'
 'GTE' 'CEN' 'AZA' 'UCL' 'SUB' 'DAL' 'GAP' 'OAT' 'DPH' 'EFU' 'OI' 'SRV'
 'TOY' 'PNU' 'KRB' 'JPM' 'AMR' 'PSFT' 'AFS' 'YNR' 'GBLX' 'WLP' 'AL' 'MAY'
 'TKR' 'ENE' 'WTHG' 'BFO' 'ABX' 'NT' 'ARC' 'MIR' 'WIN' 'TX' 'TUP' 'OWC'
 'CPQ' 'TMC' 'S' 'N' 'BGEN' 'FSR' 'BGG' 'MZ' 'GLK' 'IKN' 'NSI' 'OK' 'OKE'
 'GRA' 'LDG' 'CGP' 'PCS' 'USW' 'GPU' 'HI' 'UN' 'HRC' 'SMS' 'PBY' 'CG' 'BS'
 'RAL' 'CNXT' 'CS' 'DLX' 'RLM' 'H' 'KSU' 'SGI' 'PDG' 'SEG' 'PVN' 'NCE'
 'TOS' 'ONE' 'FJ' 'VO' 'MKG' 'NGH' 'MTC' 'UK' 'FMC' 'SMI' 'ACK' 'AET'
 'CHA' 'WB' 'COC' 'GP' 'CR' 'FPC' 'KM' 'NXTL' 'FLR' 'MEA' 'NC' 'SOTR'
 'ADPT' 'TRW' 'CNC' 'WLL' 'AGC' 'HM' 'COMS' 'UPR' 'CMCSK' 'CBS' 'QTRN'
 'CSR' 'PRD' 'UMG' 'CCK' 'JOS' 'G' 'TWX' 'RML' 'NMK' 'AM' 'PHA' 'WOR'
 'VRTS' 'SAPE' 'VC' 'CF' 'PALM' 'CIT' 'PWER' 'GX' 'BVSN' 'CPN' 'VTSS'
 'CEFT' 'JHF' 'AWE' 'IMNX' 'RATL' 'SDS' 'TAP' 'CE']


In [96]:
condition3 = df_ori['ending'].isna()
false_rows3 = df_ori[~condition3]
len(false_rows3["Ticker"])

144479

In [35]:
active_firms = df_ori[
        (df_ori['start'] <= end_of_year) &  (df_ori['ending'] >= start_of_year)
    ].drop_duplicates(subset=['permno'])

In [108]:
total2 = set(df_total2["Ticker"])
total = set(df_total["Ticker"])
set_2025 = set(df_2025["Ticker"])
print(len(total2))
print(len(total))
print(len(set_2025))

615
549
503


In [110]:
print(len(total2 - total))
interect_total_total2 = list(total2.intersection(total))
interect_total_2025 = list(set_2025.intersection(total))
interect_total2_2025 = list(set_2025.intersection(total2))
print(len(interect_total_total2))
print(len(interect_total_2025))
print(len(interect_total2_2025))

85
530
413
479


In [111]:
print(len(total2 - set_2025))
print(len(total - set_2025))
left2 = total2 - set_2025
left = total - set_2025
len(left2-left)

136
136


17

In [124]:
update_df = df_total2[~df_total2["Ticker"].isin(df_total["Ticker"])]
update_df.to_csv("update_constituents.csv", index=False)

Buliding Total SP 500 Constituents


Adding CIKs to sp500 2025 weight data

In [1]:
# import requests

# def get_cik(ticker):
#     """
#     Retrieves the CIK number for a given ticker symbol from the SEC's EDGAR system.
    
#     Parameters:
#         ticker (str): The stock ticker symbol.
    
#     Returns:
#         str: The CIK number for the given ticker symbol, or a message if not found.
#     """
#     url = 'https://www.sec.gov/files/company_tickers.json'
#     headers = {
#         'User-Agent': 'University of Edinburgh s2101369@ed.ac.uk',
#         'Accept-Encoding': 'gzip, deflate'
#     }
    
#     try:
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()
#         data = response.json()
        
#         for item in data.values():
#             if item['ticker'].lower() == ticker.lower():
#                 cik = item['cik_str']
#                 return str(cik).zfill(10)  # Pad CIK to 10 digits
        
#         return "CIK not found for the given ticker."
    
#     except requests.exceptions.RequestException as e:
#         return f"Error accessing SEC data: {e}"

# # Example usage
# ticker = 'ELV'  # Replace with the desired ticker
# cik_number = get_cik(ticker)
# print(f"CIK for {ticker}: {cik_number}")


CIK for ELV: 0001156039


In [ ]:
# path = "/Users/apple/PROJECT/Code_4_SECfilings/sp500_slickcharts.csv"
# df_2025 = pd.read_csv(path)
# df_2025 = df_2025.rename(columns={"Symbol": "Ticker"})
# df_2025["CIK"] = df_2025["Ticker"].apply(get_cik)
# df_2025['CIK'] = df_2025['CIK'].astype(str)
# df_2025.loc[9, "CIK"] = "0001067983"
# df_2025.loc[500, "CIK"] = "0000014693"

In [41]:
# df_2025.to_csv("sp500_2025_weights.csv", index=False)


In [42]:
path = "/Users/apple/PROJECT/Code_4_SECfilings/update_constituents.csv"
df_update = pd.read_csv(path)


In [43]:
df_2025.drop(columns=["Company", "Weight"], inplace=True)

In [44]:
set_2025

NameError: name 'set_2025' is not defined

In [32]:
update_df = pd.concat([df_2025, df_update], axis=0, ignore_index=True)

In [33]:
update_df.value_counts("Ticker")

,Ticker,CIK
0,AAPL,0000320193
1,NVDA,0001045810
2,MSFT,0000789019
3,AMZN,0001018724
4,META,0001326801
...,...,...
583,BG,1996862
584,BLDR,1316835
585,PODD,1145197
586,BX,1393818


In [55]:
df_update.head()

,CIK,Ticker
0,1745999,BEAM
1,798354,FI
2,216228,ITT
3,1163165,COP
4,93410,CVX


In [51]:
df_2025.head()


,Company,Ticker,Weight,CIK
0,Apple Inc.,AAPL,7.16%,320193
1,Nvidia Corp,NVDA,6.62%,1045810
2,Microsoft Corp,MSFT,5.96%,789019
3,Amazon.com Inc,AMZN,4.02%,1018724
4,"Meta Platforms, Inc. Class A",META,2.92%,1326801


In [52]:
df_total2.head()

,CIK,Ticker
0,1341439,ORCL
1,789019,MSFT
2,1113169,TROW
3,773840,HON
4,1745999,BEAM


In [67]:
only_2025 = df_2025[~df_2025["Ticker"].isin(df_total2["Ticker"])]
only_2025.drop(columns=["Company", "Weight"], inplace=True)

/var/folders/ls/rcj43y9j0d38d39d28pbbfz40000gn/T/ipykernel_4907/3482253636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_2025.drop(columns=["Company", "Weight"], inplace=True)


In [62]:
only_total = df_total[~df_total["Ticker"].isin(df_total2["Ticker"])]
only_total

,CIK,Ticker
14,101778,MRO
29,1284812,CNS
48,827099,CPWR
68,1474098,PEB
84,1675149,AA
87,1328581,BCC
89,1579684,GCI
131,1616707,W
193,1384195,REI
205,1000753,NSP


In [64]:
diff = set(only_2025["Ticker"].tolist()) - set(only_total["Ticker"].tolist())
print(len(diff))

22


In [68]:
sp500_total_constituents3 = pd.concat([df_total2, only_2025, only_total], axis=0, ignore_index=True)
sp500_total_constituents3

,CIK,Ticker
0,1341439,ORCL
1,789019,MSFT
2,1113169,TROW
3,773840,HON
4,1745999,BEAM
...,...,...
653,1691493,NU
654,1336917,UA
655,1754301,FOXA
656,861878,SRCL


In [70]:
sp500_total_constituents3.drop_duplicates(subset=["Ticker"], inplace=True)
sp500_total_constituents3.to_csv("sp500_total_constituents_final.csv", index=False)

In [72]:
update_only2025 = pd.concat([only_2025, df_update], axis=0, ignore_index=True)
update_only2025.to_csv("update_only2025.csv", index=False)